# Tutorial

Import necessary packages

In [1]:
import parallelPermutationTest as ppt
import numpy as np
import pandas as pd

# Synthetic data: Integer data

The permutation test only works on integer data, so when one has an integer dataset, one does not have to pre-process the data with a binning-procedure. Hence, one can use ppt.GreenIntCuda method.

Let's construct a synthetic dataset with integers ranging from 0 to 500, and with sample sizes of 500 elements.

In [ ]:
n_samples = 1
n = m =  500

In [ ]:
data = lambda n,n_samples : np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)])

In [ ]:
np.random.seed(42)
A,B = data(n,1), data(n,1)

The shift-algorithm implemented in the R-package Coin(https://cran.r-project.org/web/packages/coin/index.html) is probably the fastest version permutation test today. We have implemented a slightly speeded-up version of their version into Python.

In [ ]:
%time p_shift = ppt.CoinShiftInt(A,B)

Greens algorithm is a slight variation of the shift-algorithm. Unfortunately, on a single thread, it's prolonged compared to the shift-algorithm.  However, it has the perk of being parallelizable. Let's check the available thread version.

In [ ]:
%time p_green = ppt.GreenInt(A,B)

So when only a single thread is accessible, we would recommend using ppt.CoinShiftInt rather than ppt.GreenInt. However, when one has several threads, the Greens algorithm starts to shine. Let us take a look at a multithreaded version of the Greens algorithm. Here we are using an Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, which has eight threads.

In [ ]:
%time p_green_mt = ppt.GreenIntMultiThread(A,B)

Quite a large speed-up, but still only on par with ppt.CoinShiftInt. Let us use more threads! We can use GPU for this. Here we use GeForce RTX 2070.

In [ ]:
%time p_green_gpu = ppt.GreenIntCuda(A,B)

Great! We have essentially improved the run-time five times compared to ppt.CoinShiftInt.

Let us ensure that they all yield the same result.

In [ ]:
np.allclose(p_shift, p_green, p_green_mt, p_green_gpu)

# Real data: Finance

This is example is based on https://www.datacamp.com/community/tutorials/stocks-significance-testing-p-hacking.

### The claim is: "Over the past 32 years, October has been the most volatile month on average for the S&P500 and December the least volatile". 


### Let's check if this is statistically significant.

We have to download and pre-process the data.

In [ ]:
#Daily S&P500 data from 1986==>
url = "https://raw.githubusercontent.com/Patrick-David/Stocks_Significance_PHacking/master/spx.csv"
df = pd.read_csv(url,index_col='date', parse_dates=True)



In [ ]:
#To model returns we will use daily % change
daily_ret = df['close'].pct_change()
#drop the 1st value - nan
daily_ret.dropna(inplace=True)


In [ ]:
mnthly_annu = daily_ret.resample('M').std()* np.sqrt(12)

In [ ]:
dec_vol = mnthly_annu[mnthly_annu.index.month==12]
rest_vol = mnthly_annu[mnthly_annu.index.month!=12]

In [ ]:
dec_vol.head(2)

In [ ]:
(dec_vol.values.shape, rest_vol.values.shape)

Here we have float data, i.e., real values. So we can not use ppt.GreenIntCuda. We have to pre-process the data with a binning procedure. Let us take 500 bins. This procedure will map all values into 500 integer bins. 

In [ ]:
n_bins = 500

In [ ]:
%time p = ppt.GreenFloatCuda(dec_vol.values, rest_vol.values, n_bins)

In [ ]:
p

That December is the least volatile month seems not to be statistically significant.

# Real data: Biomedical data

### We want to see if there are any significant genes in breast cancer patients.

Let import the pre-processed from the Experiment 6 notebook.

In [ ]:
NotTNP_df = pd.read_csv("experiment_data/experiment6/notTNPdf")
TNP_df = pd.read_csv("experiment_data/experiment6/TNPdf")

In [ ]:
(TNP_df.shape, NotTNP_df.shape)

In [ ]:
TNP_df.head(2)

This dataset is quite large (8051 experiments) with floating values. Let us take 100 bins. However, we have to be careful, so we do not overload the GPU. Let us make a memory check.

In [ ]:
n_bins = 100

In [ ]:
ppt.GreenFloatCuda_memcheck(TNP_df.values, NotTNP_df.values, n_bins)

We need to divide our data into batches.

In [ ]:
batch_size = int(TNP_df.shape[0] / 4)

In [ ]:
%time p_values = ppt.GreenFloatCuda(TNP_df.values, NotTNP_df.values, 100, batch_size=batch_size)